In [1]:
import pandas as pd
import datetime
import os
import numpy as np

In [2]:
paramsFilename = "X:/LLFS/data_analysis/sample_metadata/LLFS_Experiment_parameters_Shipment1and2.csv"
datadir = "../../integration/skyline_documents/"
numQC = 9

In [3]:
metadata = pd.read_csv(paramsFilename)
metadata

,Batch #,Batch,Metabolomics identifier,Shipment
0,1,V,10014843_1,1
1,1,V,10014843_2,1
2,6,Q,10016673_1,1
3,6,Q,10016673_2,1
4,2,U,10041309_1,1
...,...,...,...,...
4000,43,AQ,49975486_1,2
4001,43,AQ,49975486_2,2
4002,12,F,49989098_1,1
4003,12,F,49989098_2,1


In [4]:
batches = list(set(metadata["Batch"].values))
batches.sort()
batches

['A',
 'AA',
 'AB',
 'AC',
 'AD',
 'AE',
 'AF',
 'AG',
 'AH',
 'AI',
 'AJ',
 'AK',
 'AL',
 'AM',
 'AN',
 'AO',
 'AP',
 'AQ',
 'AR',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']

In [5]:
qc_df = {}
for batch in batches:
    batchNum = metadata[metadata["Batch"] == batch]["Batch #"].values[0]
    if batch == "AM":
        tmpNumQC = 2
    elif batch == "AR":
        tmpNumQC = 7
    elif batch == "V":
        tmpNumQC = 8
    else:
        tmpNumQC = numQC
    for qc in range(1,tmpNumQC+1):
        if batch != "A":
            qc_df[len(qc_df)] = {"Batch":batch,"Batch #":batchNum,"Metabolomics identifier":"QC_Splash_0" + str(qc)}
        else:
            qc_df[len(qc_df)] = {"Batch":batch,"Batch #":batchNum,"Metabolomics identifier":"QC_0" + str(qc)}

qc_df = pd.DataFrame.from_dict(qc_df,orient="index")
metadata = pd.concat((metadata,qc_df),axis=0,ignore_index=True)
metadata

,Batch #,Batch,Metabolomics identifier,Shipment
0,1,V,10014843_1,1.0
1,1,V,10014843_2,1.0
2,6,Q,10016673_1,1.0
3,6,Q,10016673_2,1.0
4,2,U,10041309_1,1.0
...,...,...,...,...
4386,26,Z,QC_Splash_05,NaN
4387,26,Z,QC_Splash_06,NaN
4388,26,Z,QC_Splash_07,NaN
4389,26,Z,QC_Splash_08,NaN


In [6]:
folders = os.listdir(datadir)
batchMapper = {}
for batch in batches:
    for fold in folders:
        if "Batch" + batch + "_" in fold:
            batchMapper[batch] = fold

In [7]:
batchMapper

{'A': '20201119_LLFS_BatchA_pos_Lipids',
 'AA': '20210916_LLFS_BatchAA_Lipids',
 'AB': '20210920_LLFS_BatchAB_pos_Lipids',
 'AC': '20210923_LLFS_BatchAC_pos_Lipids',
 'AD': '20210927_LLFS_BatchAD_pos_Lipids',
 'AE': '20211014_LLFS_BatchAE_pos_lipids',
 'AF': '20210930_LLFS_BatchAF_pos_Lipids',
 'AG': '20211018_LLFS_BatchAG_pos_lipids',
 'AH': '20211021_LLFS_BatchAH_pos_lipids',
 'AI': '20211025_LLFS_BatchAI_pos_lipids',
 'AJ': '20211028_LLFS_BatchAJ_pos_lipids',
 'AK': '20211104_LLFS_BatchAK_pos_lipids',
 'AL': '20211108_LLFS_BatchAL_Pos_Lipids',
 'AM': '20211111_LLFS_BatchAM_pos_Lipids',
 'AN': '20211115_LLFS_BatchAN_pos_Lipids',
 'AO': '20211118_LLFS_BatchAO_pos_Lipids',
 'AP': '20211203_LLFS_BatchAP_pos_Lipids',
 'AQ': '20211213_LLFS_BatchAQ_pos_Lipids',
 'AR': '20211216_LLFS_BatchAR_pos_Lipids',
 'B': '20201123_LLFS_BatchB_pos_Lipids',
 'C': '20201130_LLFS_BatchC_pos_Lipids',
 'D': '20201203_LLFS_BatchD_pos_Lipids',
 'E': '20201207_LLFS_BatchE_pos_Lipids',
 'F': '20210121_LLFS_Batc

In [1]:
mergedDf = pd.DataFrame()
goodInds = []
filenames = ["" for _ in range(len(metadata))]
times = [datetime.datetime(1,1,1,1,1) for _ in filenames]
accounted = [False for _ in filenames]
for batch in batches:
    filt = metadata[metadata["Batch"] == batch]
    peak_areas = pd.read_csv(datadir  + "/" + batchMapper[batch] + "/peak_areas.csv",index_col=0)
    mapper = {}
    for name in peak_areas.index.values:
        if "_" in name:
            lipidType = name.split()[0]
            lipid_comp = name.split()[1]
            if "-first_list" in lipid_comp:
                lipid_comp = lipid_comp.replace("-first_list","")
            if "d" in lipid_comp:
                prefix = "d"
                lipid_comp = lipid_comp.replace("d","")
            else:
                prefix = ""
            if "_" in lipid_comp:
                lipid_comp = lipid_comp.split("_")
                lipid_comp = [x.split(":") for x in lipid_comp]
                lipid_comp = np.array(lipid_comp).astype(int)
                lipid_comp = np.sum(lipid_comp,axis=0)
                lipid_comp = str(lipid_comp[0]) + ":" + str(lipid_comp[1])

            new_name = prefix + lipidType + " " + lipid_comp
            mapper[name] = new_name
    peak_areas = peak_areas.rename(mapper=mapper,axis=0)
    metInfo = peak_areas[peak_areas.columns.values[:3]]
    peak_areas = peak_areas[peak_areas.columns.values[3:]]
    file_times = pd.read_csv(datadir + "/" + batchMapper[batch] + "/aq_time_report.csv",index_col=0)
    for index,row in file_times.iterrows():
        tmp = row["Acquired Time"]
        month = int(tmp.split()[0].split("/")[0])
        day = int(tmp.split()[0].split("/")[1])
        year = int(tmp.split()[0].split("/")[2])
        hour = int(tmp.split()[1].split(":")[0])
        minute = int(tmp.split()[1].split(":")[1])
        file_times.at[index,"Acquired Time"] = datetime.datetime(year,month,day,hour,minute)
    files = [x.replace(" Area","") for x in peak_areas.columns.values]
    for index,row in filt.iterrows():
        matches = [x for x in files if row["Metabolomics identifier"] in x and "terative" not in x]
        if len(matches) > 0:
            matches.sort(key = lambda x: file_times.at[x+".d","Acquired Time"],reverse=True)
            t = file_times.at[matches[0] + ".d","Acquired Time"]
            #print([file_times.at[x + ".d","Acquired Time"] for x in matches])
            if t > times[index]:
                times[index] = t
                filenames[index] = matches[0]
                accounted[index] = True

    mergedDf = pd.concat((mergedDf,peak_areas),axis=1,ignore_index=False)
times = [x.isoformat() for x in times]
metadata["File name lipids pos"] = filenames
metadata["Acq times for File name lipids pos"] = times
metadata["found matching File name lipids pos"] = accounted
metadata["peak area sample name"] = ["Batch" + str(batch) + "_" + id for batch,id in zip(metadata["Batch #"],metadata["Metabolomics identifier"])]

NameError: name 'pd' is not defined

In [9]:
metadata

,Batch #,Batch,Metabolomics identifier,Shipment,File name lipids pos,Acq times for File name lipids pos,found matching File name lipids pos,peak area sample name
0,1,V,10014843_1,1.0,BatchV_pL_10014843_1,2021-03-08T11:23:00,True,Batch1_10014843_1
1,1,V,10014843_2,1.0,BatchV_pL_10014843_2,2021-03-09T03:11:00,True,Batch1_10014843_2
2,6,Q,10016673_1,1.0,BatchQ_pL_10016673_1,2021-02-06T06:25:00,True,Batch6_10016673_1
3,6,Q,10016673_2,1.0,BatchQ_pL_10016673_2,2021-02-05T11:57:00,True,Batch6_10016673_2
4,2,U,10041309_1,1.0,BatchU_pL_10041309_1,2021-03-07T05:11:00,True,Batch2_10041309_1
...,...,...,...,...,...,...,...,...
4386,26,Z,QC_Splash_05,NaN,BatchZ_pL_QC_Splash_05,2021-09-14T08:24:00,True,Batch26_QC_Splash_05
4387,26,Z,QC_Splash_06,NaN,BatchZ_pL_QC_Splash_06,2021-09-15T03:28:00,True,Batch26_QC_Splash_06
4388,26,Z,QC_Splash_07,NaN,BatchZ_pL_QC_Splash_07,2021-09-15T10:32:00,True,Batch26_QC_Splash_07
4389,26,Z,QC_Splash_08,NaN,BatchZ_pL_QC_Splash_08,2021-09-15T05:36:00,True,Batch26_QC_Splash_08


In [10]:
metadata.to_csv("output_files/experiment_parameters.csv")

In [11]:
filenames = list(metadata["File name lipids pos"].values)
print(len(filenames))
goodCols = [f + " Area" for f in filenames if "Blank" not in f and f != ""]
filenames = [x.replace(" Area","") for x in goodCols]
print(len(filenames))
mergedDf = mergedDf[goodCols]
mergedDf = mergedDf.rename(mapper={c:c.replace(" Area","") for c in mergedDf.columns.values},axis=1)
mapper2 = {old:new for old,new in zip(metadata["File name lipids pos"], metadata["peak area sample name"])}
mergedDf = mergedDf.rename(mapper=mapper2,axis=1)
filenames = [mapper2[x] for x in filenames]
metInfo.to_csv("output_files/metabolite_info.csv")
mergedDf.to_csv("output_files/merged_peak_areas.csv")
metadata["Acq times for File name lipids pos"] = [datetime.datetime.fromisoformat(x) for x in metadata["Acq times for File name lipids pos"].values]
metadata = metadata.sort_values(by = "Acq times for File name lipids pos",ascending=True)
batchInfoDf = metadata.loc[metadata.index.values,["peak area sample name","Batch #"]]
batchInfoDf = batchInfoDf.rename(mapper={"Batch #":"batch"},axis=1)
batchInfoDf = batchInfoDf.set_index("peak area sample name")
batchInfoDf["order"] = list(range(1,len(batchInfoDf)+1))
batchInfoDf["sample batch order"] = list(range(1,len(batchInfoDf)+1))
batchInfoDf = batchInfoDf.loc[filenames,:]

for batch in list(set(batchInfoDf["batch"])):
    filt = batchInfoDf[batchInfoDf["batch"] == batch]
    filt = filt.sort_values(by="order",ascending=True)
    for index,i in zip(filt.index.values,range(1,len(filt)+1)):
        batchInfoDf.at[index,"sample batch order"] = i
batchInfoDf.to_csv("output_files/batchInfo.csv")


4391
4391


In [12]:
mergedDf

,Batch1_10014843_1,Batch1_10014843_2,Batch6_10016673_1,Batch6_10016673_2,Batch2_10041309_1,Batch2_10041309_2,Batch5_10094670_1,Batch5_10111300_1,Batch5_10187532_1,Batch1_10189066_1,...,Batch25_QC_Splash_09,Batch26_QC_Splash_01,Batch26_QC_Splash_02,Batch26_QC_Splash_03,Batch26_QC_Splash_04,Batch26_QC_Splash_05,Batch26_QC_Splash_06,Batch26_QC_Splash_07,Batch26_QC_Splash_08,Batch26_QC_Splash_09
Molecule Name,,,,,,,,,,,,,,,,,,,,,
ACar 10:0,149105,200098,435899,683371,1011105,556529,700669,312429,481107,442556,...,315552,584092,545542,538349,523064,525163,508830,500922,507525,513969
ACar 10:1,176548,139917,160724,264973,351125,276781,425940,328645,338432,185556,...,285799,419669,387250,399542,390043,380455,366510,371451,369958,363472
ACar 11:1,39034,52352,133829,198240,91081,91309,75309,148249,55932,123782,...,57875,84696,85318,90189,85251,78814,81350,82890,79305,81842
ACar 12:1,84743,131139,192108,268217,267817,214025,184265,250397,220878,133259,...,156049,274531,261350,246369,250538,245298,242270,247433,242518,244098
ACar 13:1,20182,14482,73489,86928,36843,32857,38915,62791,27343,39638,...,55432,104850,93097,93382,96662,88037,89872,80475,88320,83368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TG 54:5,71775624,126603360,52748220,50737248,97381632,122939032,65183088,82858112,117587520,81477144,...,121045824,135072384,127887752,125873912,123744208,122787912,121659376,120726488,120484384,119560984
TG 56:2,6202205,5269447,2502554,3360935,1943818,5419461,1422488,1238633,4336014,3020522,...,1839037,2192036,2021684,1982289,1962989,1939815,1889872,1897176,1882769,1839760
TG 56:5,21668610,41741428,16275522,15109690,29192584,31285376,21698528,26045362,27166190,31405836,...,23828002,27037688,25754670,25228782,24770832,24628168,24470712,23939484,23979940,23792316


In [13]:
batchInfoDf

,batch,order,sample batch order
peak area sample name,,,
Batch1_10014843_1,1,2120,18
Batch1_10014843_2,1,2130,28
Batch6_10016673_1,6,1682,83
Batch6_10016673_2,6,1660,61
Batch2_10041309_1,2,2088,87
...,...,...,...
Batch26_QC_Splash_05,26,2547,42
Batch26_QC_Splash_06,26,2574,69
Batch26_QC_Splash_07,26,2600,95


In [14]:
metadata

,Batch #,Batch,Metabolomics identifier,Shipment,File name lipids pos,Acq times for File name lipids pos,found matching File name lipids pos,peak area sample name
4005,11,A,QC_01,NaN,BatchA_pL_QC_01,2020-11-19 02:42:00,True,Batch11_QC_01
1678,11,A,27754655_1,1.0,BatchA_pL_27754655_1,2020-11-19 03:14:00,True,Batch11_27754655_1
1957,11,A,29898244_1,1.0,BatchA_pL_29898244_1,2020-11-19 03:47:00,True,Batch11_29898244_1
860,11,A,21170212_1,1.0,BatchA_pL_21170212_1,2020-11-19 04:19:00,True,Batch11_21170212_1
1054,11,A,22647044_1,1.0,BatchA_pL_22647044_1,2020-11-19 04:52:00,True,Batch11_22647044_1
...,...,...,...,...,...,...,...,...
3356,44,AR,41146598_2,2.0,BatchAR_pL_41146598_2,2021-12-18 07:21:00,True,Batch44_41146598_2
3460,44,AR,42522129_1,2.0,BatchAR_pL_42522129_1,2021-12-18 07:53:00,True,Batch44_42522129_1
4166,44,AR,QC_Splash_07,NaN,BatchAR_pL_QC_Splash_07,2021-12-18 08:26:00,True,Batch44_QC_Splash_07
3945,44,AR,49043375_2,2.0,BatchAR_pL_49043375_2,2021-12-18 12:17:00,True,Batch44_49043375_2
